<a href="https://colab.research.google.com/github/dona-frj/semantic/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
! unzip /content/drive/MyDrive/aquaint.zip -d'/content/data'

Archive:  /content/drive/MyDrive/aquaint.zip
replace /content/data/aquaint/RawText/APW19980603_0791.htm? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
!unzip aquaint.zip

unzip:  cannot find or open aquaint.zip, aquaint.zip.zip or aquaint.zip.ZIP.


In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize , sent_tokenize
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum ,strip_short
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
pip install tqdm

/////////////////////////// creat data to list

In [9]:
import os
from tqdm import tqdm
path = '/content/data/aquaint/RawText/'
list_txt=[]
tokenized_sent = [] #من اجل تخزين الجمل الماخوذة من النصوص
for txt  in os.listdir(path):
  f = open(os.path.join(path,txt), "r")         
  _txt= f.read()
  _txt=strip_multiple_whitespaces(_txt)
  _txt= remove_stopwords(_txt)
  _txt= sent_tokenize(_txt) 
  for i in _txt:
    t1=strip_non_alphanum(i) 
    t1=strip_short(t1,minsize=3)
    t2=remove_stopwords(t1)
    
  
    list_txt.append(t2)
list_txt

['LONDON Nov',
 'The apparent settlement came days talks involving West Indies players officials West Indies South Africa cricket boards',
 'The crisis came head Wednesday West Indies Cricket Board fired superstar batsman Brian Lara captain Carl Hooper vice captain',
 'The refused travel South Africa demanding better pay conditions tour',
 'Team members insisted Lara Hooper reinstated settlement',
 'Jimmy Adams treasurer players assocation negotiated behalf players Monday Pat Rousseau president West Indies Cricket Board',
 'Ali Bacher head South African cricket directly involved Monday talks sitting sidelines Sunday',
 'West Indies tour manager Clive Lloyd WICB member Joel Garner involved talks began Friday',
 'Progress reported weekend announcement sponsorship deal offered South African wicketkeeper David Richardson runs sports marketing firm Johannesburg',
 'Earnings sponsorship proposal apparently help meet players demands better pay reported range dlrs 000 000 South African tour',


In [10]:
for s in list_txt:
    tokenized_sent.append(word_tokenize(s.lower()))
len(tokenized_sent)
# tokenized_sent

517

In [11]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
tagged_data

[TaggedDocument(words=['london', 'nov'], tags=[0]),
 TaggedDocument(words=['the', 'apparent', 'settlement', 'came', 'days', 'talks', 'involving', 'west', 'indies', 'players', 'officials', 'west', 'indies', 'south', 'africa', 'cricket', 'boards'], tags=[1]),
 TaggedDocument(words=['the', 'crisis', 'came', 'head', 'wednesday', 'west', 'indies', 'cricket', 'board', 'fired', 'superstar', 'batsman', 'brian', 'lara', 'captain', 'carl', 'hooper', 'vice', 'captain'], tags=[2]),
 TaggedDocument(words=['the', 'refused', 'travel', 'south', 'africa', 'demanding', 'better', 'pay', 'conditions', 'tour'], tags=[3]),
 TaggedDocument(words=['team', 'members', 'insisted', 'lara', 'hooper', 'reinstated', 'settlement'], tags=[4]),
 TaggedDocument(words=['jimmy', 'adams', 'treasurer', 'players', 'assocation', 'negotiated', 'behalf', 'players', 'monday', 'pat', 'rousseau', 'president', 'west', 'indies', 'cricket', 'board'], tags=[5]),
 TaggedDocument(words=['ali', 'bacher', 'head', 'south', 'african', 'cric

In [12]:
model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)

'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''

## Print model vocabulary
model.wv.vocab
test_doc = word_tokenize("We to want projects success paper inspection to desired results added".lower())
test_doc_vector = model.infer_vector(test_doc)
model.docvecs.most_similar(positive = [test_doc_vector])


[(188, 0.934506356716156),
 (189, 0.9338201880455017),
 (276, 0.9285963177680969),
 (18, 0.9275515079498291),
 (332, 0.9239315986633301),
 (374, 0.9228423237800598),
 (187, 0.9217140078544617),
 (248, 0.9205392599105835),
 (50, 0.9202824831008911),
 (385, 0.9197394251823425)]

In [13]:
pip install transformers 

In [14]:
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers
## for data
import json
import pandas as pd
import numpy as np

In [15]:
txt = list_txt
## bert tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
## bert model
nlp = transformers.TFBertModel.from_pretrained('bert-base-uncased')
## return hidden layer with embeddings
input_ids = np.array(tokenizer.encode(txt))[None,:500]  
embedding = nlp(input_ids)
embedding[0][0]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result 

<tf.Tensor: shape=(500, 768), dtype=float32, numpy=
array([[ 0.5163425 ,  0.08618787, -0.2219882 , ..., -0.38287237,
         0.46872312, -0.3554358 ],
       [-0.3069918 ,  0.5394577 , -1.1727283 , ..., -0.46522778,
         0.1972778 , -0.19726875],
       [-0.34485194,  0.5360682 , -1.1604458 , ..., -0.4882096 ,
         0.16486694, -0.19340603],
       ...,
       [ 0.5624937 ,  0.21455921, -0.4395885 , ..., -0.40097788,
         0.2475631 , -0.43680915],
       [ 0.5452322 ,  0.21563193, -0.4388248 , ..., -0.39957795,
         0.2571793 , -0.4173339 ],
       [ 0.54675156,  0.32149938, -0.44606358, ..., -0.34326655,
         0.2237314 , -0.35612074]], dtype=float32)>

In [ ]:
## distil-bert tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

In [17]:
lst_dics = tagged_data

## print the first one
lst_dics[0]
## create dtf
dtf = pd.DataFrame(lst_dics)
## filter categories
# dtf = dtf[ dtf["category"].isin(['ENTERTAINMENT','POLITICS','TECH']) ][["category","headline"]]
## rename columns
dtf = dtf.rename(columns={"category":"y", "headline":"text"})
## print 10 random rows
dtf.sample(10)

,words,tags
183,"[norbert, otto, federal, lawmaker, erfurt, dem...",[183]
346,"[later, joined, cpi, remained, member, 1978]",[346]
434,"[only, small, number, killings, solved]",[434]
134,"[quoted, saleh, saying, discuss, iran, trade, ...",[134]
67,"[about, 100, palestinian, airport, employees, ...",[67]
468,"[the, storm, roared, texas, oklahoma, thursday...",[468]
199,"[with, crime, increasingly, sophisticated, kan...",[199]
388,"[and, bids, accepted, customers, registered, c...",[388]
405,"[other, cult, leaders, charged, gassing, slayi...",[405]
277,"[ambitious, modern, wide, ranging, efficient, ...",[277]


In [18]:
## inputs
idx = layers.Input((50), dtype="int32", name="input_idx")
masks = layers.Input((50), dtype="int32", name="input_masks")
## pre-trained bert with config
config = transformers.DistilBertConfig(dropout=0.2, 
           attention_dropout=0.2)
config.output_hidden_states = False
nlp = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)
bert_out = nlp(idx, attention_mask=masks)[0]
## fine-tuning
x = layers.GlobalAveragePooling1D()(bert_out)
x = layers.Dense(64, activation="relu")(x)
y_out = layers.Dense(len(np.unique(list_txt)), 
                     activation='softmax')(x)
## compile
model = models.Model([idx, masks], y_out)
for layer in model.layers[:3]:
    layer.trainable = False
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
model.summary()


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

Collect the **resault** from the last unworkable part!! 
////*Help in this* ///

In [19]:
## encode y
dic_y_mapping = {n:label for n,label in enumerate(np.unique(y_train))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
y_train = np.array([inverse_dic[y] for y in y_train])
## train
training = model.fit(x=X_train, y=y_train, batch_size=64, 
                     epochs=1, shuffle=True, verbose=1, 
                     validation_split=0.3)
## test
predicted_prob = model.predict(X_test)
predicted = [dic_y_mapping[np.argmax(pred)] for pred in 
             predicted_prob]

NameError: ignored

In [24]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import sent_tokenize

from sentence_transformers import SentenceTransformer

base_document = lst_dics
documents = ["This is the collection of documents"]

def process_bert_similarity():
	# This will download and load the pretrained model offered by UKPLab.
	model = SentenceTransformer('bert-base-nli-mean-tokens')

	# Although it is not explicitly stated in the official document of sentence transformer, the original BERT is meant for a shorter sentence. We will feed the model by sentences instead of the whole documents.
	sentences = base_document
	base_embeddings_sentences = model.encode(sentences)
	base_embeddings = np.mean(np.array(base_embeddings_sentences), axis=0)

	vectors = []
	for i, document in enumerate(documents):

		sentences = sent_tokenize(document)
		embeddings_sentences = model.encode(sentences)
		embeddings = np.mean(np.array(embeddings_sentences), axis=0)

		vectors.append(embeddings)

		# print("making vector at index:", i)

	scores = cosine_similarity([base_embeddings], vectors).flatten()

	highest_score = 0
	highest_score_index = 0
	for i, score in enumerate(scores):
		if highest_score < score:
			highest_score = score
			highest_score_index = i

	most_similar_document = documents[highest_score_index]
	print("Most similar document by BERT with the score:", most_similar_document, highest_score)

process_bert_similarity()

making vector at index: 0
Most similar document by BERT with the score: This is the collection of documents 0.5863608


In [ ]:
lst_dics = tagged_data

## print the first one
lst_dics[0]
## create dtf
dtf = pd.DataFrame(lst_dics)
## filter categories
# dtf = dtf[ dtf["category"].isin(['ENTERTAINMENT','POLITICS','TECH']) ][["category","headline"]]
## rename columns
dtf = dtf.rename(columns={"category":"y", "headline":"text"})
## print 10 random rows
dtf.sample(10)

,words,tags
183,"[norbert, otto, federal, lawmaker, erfurt, dem...",[183]
346,"[later, joined, cpi, remained, member, 1978]",[346]
434,"[only, small, number, killings, solved]",[434]
134,"[quoted, saleh, saying, discuss, iran, trade, ...",[134]
67,"[about, 100, palestinian, airport, employees, ...",[67]
468,"[the, storm, roared, texas, oklahoma, thursday...",[468]
199,"[with, crime, increasingly, sophisticated, kan...",[199]
388,"[and, bids, accepted, customers, registered, c...",[388]
405,"[other, cult, leaders, charged, gassing, slayi...",[405]
277,"[ambitious, modern, wide, ranging, efficient, ...",[277]


In [ ]:
# pip install sentence_transformers